In [1]:
import pandas as pd
import base64

In [2]:
d_path = '/raid/AISSEL/Hamed/datasets'
df = pd.read_pickle(f'{d_path}/google_images.pk')
df

,topic,caption,context,tag,query_gt,gt_coverage,image
0,309,Police are calling for public assistance after...,Police areseeking public help after a man alle...,"[pedestrian:1, close:0, rail:0, train:0, railw...","[pedestrian, close, rail, train, railway, line]",0.333333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,309,Passengers are waiting to bord train.,The Victorian Minister for Transport Infrastru...,"[pedestrian:1, close:0, rail:1, train:1, railw...","[pedestrian, close, rail, train, railway, line]",0.833333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,309,"Box Hill’s $754,000 footbridge across the tracks",This is the story of the $56.5 million grade s...,"[pedestrian:0, close:0, rail:1, train:1, railw...","[pedestrian, close, rail, train, railway, line]",0.500000,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,309,Metro Trains have so far been unable to say wh...,Metro Trains have so far been unable to say wh...,"[pedestrian:1, close:0, rail:0, train:0, railw...","[pedestrian, close, rail, train, railway, line]",0.333333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,309,"The Electric Tilt Train, the fastest train in ...",High-speed rail in Australia has been under in...,"[pedestrian:1, close:1, rail:1, train:1, railw...","[pedestrian, close, rail, train, railway, line]",0.833333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
...,...,...,...,...,...,...,...
306,99,ARCE Students Place Second in EERI Seismic Des...,A team of Cal Poly architectural engineering (...,"[pedestrian:0, structure:1, concrete:0, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.333333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
307,99,Timber Strong Provides Real-World Engineering ...,"For the last three years, Simpson Strong-Tie h...","[pedestrian:1, structure:0, concrete:1, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.222222,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
308,99,"Students Compete to Design, Build Timber Struc...",Hammer strikes echoed through the sunny plaza ...,"[pedestrian:1, structure:1, concrete:1, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.666667,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
309,99,Cal Poly Architectural Engineering Students Wi...,SAN LUIS OBISPO – Six Cal Poly architectural e...,"[pedestrian:1, structure:1, concrete:1, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.888889,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [3]:
%cd OFA

/home/barzamini/ofa/OFA


In [4]:
import torch
import numpy as np
from fairseq import utils,tasks
from fairseq import checkpoint_utils
from utils.eval_utils import eval_step
from tasks.mm_tasks.caption import CaptionTask
from models.ofa import OFAModel
from PIL import Image

# Register caption task
tasks.register_task('caption',CaptionTask)

# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False

In [5]:
# Load pretrained ckpt & config
#large model
# cp_path = f'checkpoints/caption.pt'
#base model
# cp_path = 'checkpoints/caption_base_best.pt'

#base retrained model by missinig topics
cp_path = 'checkpoints/caption_stage2_best_base.pt'

overrides={"bpe_dir":"utils/BPE", "eval_cider":False, "beam":5, "max_len_b":16, "no_repeat_ngram_size":3, "seed":7}
models, cfg, task = checkpoint_utils.load_model_ensemble_and_task(
        utils.split_paths(cp_path),
        arg_overrides=overrides
    )

# Move models to GPU
for model in models:
    model.eval()
    if use_fp16:
        model.half()
    if use_cuda and not cfg.distributed_training.pipeline_model_parallel:
        model.cuda()
    model.prepare_for_inference_(cfg)

# Initialize generator
generator = task.build_generator(models, cfg.generation)

2022-05-06 18:12:55 | INFO | tasks.ofa_task | source dictionary: 59457 types
2022-05-06 18:12:55 | INFO | tasks.ofa_task | target dictionary: 59457 types
/home/barzamini/.conda/envs/ofa/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755861072/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
# Image transform
from torchvision import transforms
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

patch_resize_transform = transforms.Compose([
    lambda image: image.convert("RGB"),
    transforms.Resize((cfg.task.patch_image_size, cfg.task.patch_image_size), interpolation=Image.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Text preprocess
bos_item = torch.LongTensor([task.src_dict.bos()])
eos_item = torch.LongTensor([task.src_dict.eos()])
pad_idx = task.src_dict.pad()
def encode_text(text, length=None, append_bos=False, append_eos=False):
    s = task.tgt_dict.encode_line(
        line=task.bpe.encode(text),
        add_if_not_exist=False,
        append_eos=False
    ).long()
    if length is not None:
        s = s[:length]
    if append_bos:
        s = torch.cat([bos_item, s])
    if append_eos:
        s = torch.cat([s, eos_item])
    return s

# Construct input for caption task
def construct_sample(image: Image):
    patch_image = patch_resize_transform(image).unsqueeze(0)
    patch_mask = torch.tensor([True])
    src_text = encode_text(" what does the image describe?", append_bos=True, append_eos=True).unsqueeze(0)
    src_length = torch.LongTensor([s.ne(pad_idx).long().sum() for s in src_text])
    sample = {
        "id":np.array(['42']),
        "net_input": {
            "src_tokens": src_text,
            "src_lengths": src_length,
            "patch_images": patch_image,
            "patch_masks": patch_mask
        }
    }
    return sample
  
# Function to turn FP32 to FP16
def apply_half(t):
    if t.dtype is torch.float32:
        return t.to(dtype=torch.half)
    return t

/home/barzamini/.conda/envs/ofa/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  
/home/barzamini/.conda/envs/ofa/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [7]:
def get_caption(b64_img):
    image_64_decode = base64.b64decode(b64_img) 
    image_result = open('test.jpg', 'wb')
    image_result.write(image_64_decode)
    image = Image.open('test.jpg')

    # Construct input sample & preprocess for GPU if cuda available
    sample = construct_sample(image)
    sample = utils.move_to_cuda(sample) if use_cuda else sample
    sample = utils.apply_to_sample(apply_half, sample) if use_fp16 else sample
    with torch.no_grad():
        result, scores = eval_step(task, generator, models, sample)
#     print(f'\n {result}\n')
    return result[0]['caption']

In [8]:
df['ofa_caption'] = df['image'].apply(get_caption)

/home/barzamini/.conda/envs/ofa/lib/python3.7/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/home/barzamini/ofa/OFA/models/sequence_generator.py:698: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = bbsz_idx // beam_size


In [9]:
df

,topic,caption,context,tag,query_gt,gt_coverage,image,ofa_caption
0,309,Police are calling for public assistance after...,Police areseeking public help after a man alle...,"[pedestrian:1, close:0, rail:0, train:0, railw...","[pedestrian, close, rail, train, railway, line]",0.333333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,people wearing face masks walk outside a train...
1,309,Passengers are waiting to bord train.,The Victorian Minister for Transport Infrastru...,"[pedestrian:1, close:0, rail:1, train:1, railw...","[pedestrian, close, rail, train, railway, line]",0.833333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,a crowd of people waiting for the train to arr...
2,309,"Box Hill’s $754,000 footbridge across the tracks",This is the story of the $56.5 million grade s...,"[pedestrian:0, close:0, rail:1, train:1, railw...","[pedestrian, close, rail, train, railway, line]",0.500000,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,a train passes under the pedestrian bridge ove...
3,309,Metro Trains have so far been unable to say wh...,Metro Trains have so far been unable to say wh...,"[pedestrian:1, close:0, rail:0, train:0, railw...","[pedestrian, close, rail, train, railway, line]",0.333333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,a train arrives at a station in the city of vi...
4,309,"The Electric Tilt Train, the fastest train in ...",High-speed rail in Australia has been under in...,"[pedestrian:1, close:1, rail:1, train:1, railw...","[pedestrian, close, rail, train, railway, line]",0.833333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,A yellow train on the tracks at the station
...,...,...,...,...,...,...,...,...
306,99,ARCE Students Place Second in EERI Seismic Des...,A team of Cal Poly architectural engineering (...,"[pedestrian:0, structure:1, concrete:0, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.333333,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,people working on the construction of a wall
307,99,Timber Strong Provides Real-World Engineering ...,"For the last three years, Simpson Strong-Tie h...","[pedestrian:1, structure:0, concrete:1, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.222222,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,A group of people posing for a picture in fron...
308,99,"Students Compete to Design, Build Timber Struc...",Hammer strikes echoed through the sunny plaza ...,"[pedestrian:1, structure:1, concrete:1, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.666667,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,pedestrians walking on the sidewalk in front o...
309,99,Cal Poly Architectural Engineering Students Wi...,SAN LUIS OBISPO – Six Cal Poly architectural e...,"[pedestrian:1, structure:1, concrete:1, masonr...","[pedestrian, structure, concrete, masonry, ins...",0.888889,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...,A group of people standing on top of a box


In [10]:
d_path = '/raid/AISSEL/Hamed/datasets'
# df.to_pickle(f'{d_path}/google_images_with_prediction.pk')
# df.to_pickle(f'{d_path}/google_images_with_prediction_base.pk')
df.to_pickle(f'{d_path}/google_images_with_prediction_base_missed.pk')

In [11]:
! ls

checkpoints	evaluate.py  __pycache__		tasks
checkpoints.md	examples     README_EncouragingLoss.md	test.jpg
colab.md	fairseq      README.md			trainer.py
criterions	LICENSE      requirements.txt		train.py
data		models	     run_scripts		transformers.md
datasets.md	ofa_module   spaces.md			utils
